In [25]:
import pandas as pd

df = pd.read_csv("../../../iPinyou/make-ipinyou-data-master/all/test.log.txt",header=0,sep='\t',index_col=False)

In [26]:
print df[["click","weekday","hour","advertiser"]].head()

   click  weekday  hour  advertiser
0      0        6     0        3358
1      0        6     0        3427
2      0        6     0        3427
3      0        6     0        3386
4      0        6     0        3358


In [27]:
###############Campaign cvr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np


df_sub_cvr =  df[["click","weekday","hour","advertiser"]]
df_add_features = pd.DataFrame(df_sub_cvr.groupby(['advertiser','weekday','hour','click']).size().reset_index(name='count'))
print df_add_features.head()

   advertiser  weekday  hour  click  count
0        1458        0     0      0   8499
1        1458        0     0      1     11
2        1458        0     1      0   5230
3        1458        0     1      1      1
4        1458        0     2      0   1422


In [28]:
#i = 0
for name, group in df_add_features.groupby(['advertiser','weekday','hour']): 
    #i += 1
    #print name[1]

    counts = group['count'].values
    #print group
    #print group['purchase'].values
    
    if ((len(counts) == 1) & (group['click'].values[0] == 0)):
        cvr_h = 0
    elif ((len(counts) == 1) & (group['click'].values[0] == 1)):
        cvr_h = 1
    else:
        cvr_h = float(counts[1])/float(counts.sum())
    
    df.ix[(df.advertiser==name[0]) & (df.weekday == name[1])
              & (df.hour == name[2]), 'cvr_h'] = cvr_h

    #if i == 10:
    #    break

In [29]:
# compare the previous two hours (t-1, t-2) cvr
from itertools import izip
from math import log
import sys
print df.shape[0]

cvr_diff_df = df.drop_duplicates(['advertiser','weekday','hour'])[['advertiser','weekday','hour','cvr_h']]
print cvr_diff_df.shape[0]
print cvr_diff_df.head()

  
epsilon = np.finfo(float).eps

def myfunc(group):
    group = group.sort_values(['weekday', 'hour'], ascending=[True, True])
    for i in range(1,2):
        diff_cvr = []
        diff_cvr.extend(np.zeros(i+1,dtype=np.int))
        cvr_col = group['cvr_h'].values
             
        for x, y in zip(cvr_col,cvr_col[1:]):
            if y - x > 0.0:
                diff_cvr.append(1)
            elif y - x == 0.0:
                diff_cvr.append(0)
            else: 
                diff_cvr.append(-1)
                
        col_name = 'diff_cvr_' + str(i)
        group[col_name] = pd.Series(diff_cvr[:-i], index=group.index)
        #print diff_cvr
        #group['cvr_ratio'] = pd.Series(ratio_cvr[:-1], index=group.index)
    #print group
    return group


cvr_diff_result = cvr_diff_df.groupby(['advertiser']).apply(myfunc)
print cvr_diff_result.tail()
#print cvr_diff_result[cvr_diff_result['cvr_ratio'] != 1].head()
#print cvr_diff_result[cvr_diff_result['idcampaign'] == 90]

4100716
556
       advertiser  weekday  hour     cvr_h
0            3358        6     0  0.001024
1            3427        6     0  0.000736
3            3386        6     0  0.000458
27           1458        6     0  0.000527
91678        1458        6     1  0.002396
                   advertiser  weekday  hour     cvr_h  diff_cvr_1
advertiser                                                        
3476       595664        3476        6    19  0.000917           1
           641556        3476        6    20  0.000737          -1
           687205        3476        6    21  0.000384          -1
           729103        3476        6    22  0.000348          -1
           781812        3476        6    23  0.000468          -1


In [30]:
df_merge = pd.merge(df,cvr_diff_result[['advertiser','weekday','hour','diff_cvr_1']], 
                    how='left', on=['advertiser','weekday','hour'])
print df_merge.tail()
print df_merge.shape[0]

         click  weekday  hour                             bidid  \
4100711      0        0    16   b74bfc1f007548e5161364195364fd0   
4100712      0        0    11   11544056b9ba1dac7f1a8546492a9a7   
4100713      0        0    11  f7e3b0a2187f9498c3b6d8c347599b91   
4100714      0        0    12  cbde18f2f6da699627cdc3a763270ef2   
4100715      0        0    17  e04aeab44180041b55325e9965c5c502   

                 timestamp  logtype                ipinyouid       useragent  \
4100711  20131028164004071        1             DAJHEK8eff73  windows_chrome   
4100712  20131028114100659        1             DAJJv60zgdbX  windows_chrome   
4100713  20131028114406381        1             DALA7Q6Vegpi      windows_ie   
4100714  20131028124001962        1             DALAWCCsgwfR      windows_ie   
4100715  20131028172306685        1  20111215121025025818971      windows_ie   

                    IP  region    ...      creative bidprice payprice keypage  \
4100711    113.72.95.*     216    .

In [31]:
df_merge.to_csv('../../../iPinyou/ipinyou-data-addflag/ipinyou_testall.flag.csv',index=False, sep="\t")